In [ ]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, IntSlider
from google.colab import output
output.enable_custom_widget_manager()
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [ ]:
contacts_df = pd.read_excel(
    'Processed_Contacts_df.xlsx',
    dtype={'Id': str}
)
calls_df = pd.read_excel(
    'Processed_Calls_df.xlsx',
    dtype={'Id': str, "CONTACTID": str}
)
spend_df = pd.read_excel('Processed_Spend_df.xlsx')
deals_df = pd.read_excel(
    'Processed_Deals_df.xlsx',
    dtype={'Id': str, 'Contact Name': str}
)

# **Анализ временных рядов**


## 1. Проанализируйте тенденцию создания сделок с течением времени и их связь сo звонками.

**Анализ основных трендов сделок и звонков с течением времени**

In [ ]:
contacts_df['Created Date'] = contacts_df['Created Time'].dt.date
calls_df['Call Date'] = calls_df['Call Start Time'].dt.date
deals_df['Deal Created Date'] = deals_df['Created Time'].dt.date

deals_per_day = deals_df.groupby('Deal Created Date').size().reset_index(
    name='Deals Count'
)
calls_per_day = calls_df.groupby('Call Date').size().reset_index(
    name='Calls Count'
)
contacts_per_day = contacts_df.groupby('Created Date').size().reset_index(
    name='Contacts Count'
)
contacts_per_day['Created Date'] = pd.to_datetime(contacts_per_day['Created Date'])

spend_per_day = spend_df.groupby('Date').agg(
    {'Spend': 'sum', 'Impressions': 'sum', 'Clicks': 'sum'}
).reset_index()

time_series_df = pd.merge(
    deals_per_day,
    calls_per_day,
    left_on='Deal Created Date',
    right_on='Call Date',
    how='outer'
).dropna()
time_series_df.rename(columns={'Deal Created Date': 'Date'}, inplace=True)

time_series_df['Date'] = pd.to_datetime(time_series_df['Date'], errors='raise')
spend_per_day['Date'] = pd.to_datetime(spend_per_day['Date'])

time_series_df = pd.merge(
    time_series_df,
    contacts_per_day,
    left_on='Date',
    right_on='Created Date',
    how='outer'
).dropna()
time_series_df = pd.merge(
    time_series_df,
    spend_per_day,
    on='Date',
    how='outer'
).dropna()
time_series_df.drop(columns=['Created Date'], inplace=True)

numeric_cols = time_series_df.select_dtypes(include=['number']).columns

weekly_time_series_df = time_series_df.resample(
    'W', on='Date'
)[numeric_cols].sum().reset_index()

In [ ]:
colors = ['#08306b', '#60a7d2', '#cbdef1', '#6080b2']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=weekly_time_series_df['Date'],
    y=weekly_time_series_df['Deals Count'],
    mode='lines+markers',
    name='Количество сделок (нед.)',
    line=dict(color=colors[0])
))

deals_trend = px.scatter(
    weekly_time_series_df, x='Date', y='Deals Count',
    trendline='ols', template='plotly_white'
)
trend_data = deals_trend.data[1]
trend_data.line.color = colors[3]
fig.add_trace(trend_data)

fig.add_trace(go.Scatter(
    x=weekly_time_series_df['Date'],
    y=weekly_time_series_df['Calls Count'],
    mode='lines+markers',
    name='Количество звонков (нед.)',
    line=dict(color=colors[1])
))

calls_trend = px.scatter(
    weekly_time_series_df, x='Date', y='Calls Count',
    trendline='ols', template='plotly_white'
)
trend_data = calls_trend.data[1]
trend_data.line.color = colors[3]
fig.add_trace(trend_data)

fig.update_layout(
    title=dict(
        text='Тренды сделок и звонков по неделям',
        x=0.5,
        font=dict(size=20)
    ),
    yaxis_title='Количество',
    legend_title='Показатель',
    xaxis=dict(tickangle=45),
    template='plotly_white'
)

fig.show()

*Количество сделок растет по экспоненциальному тренду, что видно по трендовой линии. Это может говорить об увеличении количества клиентов, улучшении маркетинговых стратегий или сезонности.
Количество звонков также увеличивается, но гораздо медленнее. Колебание количества сделок может быть связано с маркетинговыми кампаниями, акциями или внешними факторами.Видно, что на определенных участках графика (например, в январе и мае 2024) количество сделок резко падает. Возможно, это связано с праздничными периодами или изменением спроса.*

**Анализ динамики ключевых показателей по неделям**

In [ ]:
colors = ['#08306b', '#60a7d2', '#ff910d', '#9c7648']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=weekly_time_series_df['Date'],
    y=weekly_time_series_df['Deals Count'],
    mode='lines+markers',
    name='Количество сделок',
    line=dict(color=colors[2])
))

fig.add_trace(go.Scatter(
    x=weekly_time_series_df['Date'],
    y=weekly_time_series_df['Calls Count'],
    mode='lines+markers',
    name='Количество звонков',
    line=dict(color=colors[1])
))

if 'Contacts Count' in weekly_time_series_df.columns:
    fig.add_trace(go.Scatter(
        x=weekly_time_series_df['Date'],
        y=weekly_time_series_df['Contacts Count'],
        mode='lines+markers',
        name='Количество контактов',
        line=dict(color=colors[0])
    ))

if 'Spend' in weekly_time_series_df.columns:
    fig.add_trace(go.Scatter(
        x=weekly_time_series_df['Date'],
        y=weekly_time_series_df['Spend'],
        mode='lines+markers',
        name='Расходы на рекламу',
        line=dict(color=colors[3], dash='dot')
    ))

fig.update_layout(
    title=dict(
        text='Динамика ключевых показателей по неделям',
        x=0.5,
        font=dict(size=24)
    ),
    yaxis_title='Количество / Расходы',
    legend_title='Показатель',
    xaxis=dict(tickangle=45),
    template='plotly_white'
)

fig.show()

 *Наибольшая активность приходится на весну 2024 года, что коррелирует с пиком расходов на рекламу. Это может быть связано с праздничным сезоном или маркетинговыми кампаниями.*
*Высокие расходы на рекламу в декабре 2023 – январе 2024 года (до 5000) привели к росту контактов, звонков и сделок, что указывает на положительный эффект рекламных вложений в этот период. Однако весенний спад расходов сопровождался снижением всех метрик.*
 *Количество звонков (до 3000) значительно превышает количество сделок (до 1500) и контактов (до 2500), что может указывать на низкую конверсию звонков в сделки. Стабильное количество сделок  при росте звонков и расходов с апреля 2024 года может свидетельствовать о неэффективности обработки звонков или недостаточной квалификации лидов.*
*Восстановление активности к июлю 2024 года при умеренных расходах говорит о возможности дальнейшего роста при увеличении рекламного бюджета.*
*Необходимо увеличить расходы на рекламу в периоды спада для поддержки активности.*

**Анализ зависимости конверсии и среднего количества звонков на сделку**

In [ ]:
calls_df['Call Start Time'] = pd.to_datetime(calls_df['Call Start Time'])
deals_df['Created Time'] = pd.to_datetime(deals_df['Created Time'])

calls_df['Call Date'] = calls_df['Call Start Time'].dt.date
deals_df['Deal Created Date'] = deals_df['Created Time'].dt.date

weekly_calls = calls_df.groupby('Call Date').agg({'Id': 'count'}).reset_index()
weekly_calls.rename(
    columns={'Id': 'Calls Count', 'Call Date': 'Date'}, inplace=True
)
weekly_deals = deals_df.groupby('Deal Created Date').agg({'Id': 'count'}).\
                  reset_index()
weekly_deals.rename(
    columns={'Id': 'Deals Count', 'Deal Created Date': 'Date'}, inplace=True
)
merged_df = pd.merge(weekly_calls, weekly_deals, on='Date', how='left').fillna(0)

merged_df['Conversion Rate'] = (
    merged_df['Deals Count'] / merged_df['Calls Count']
) * 100
merged_df['Conversion Rate'].replace([float('inf'), float('nan')], 0,
                                     inplace=True)

merged_df['Calls per Deal'] = (
    merged_df['Calls Count'] / merged_df['Deals Count']
)
merged_df['Calls per Deal'].replace([float('inf'), float('nan')], 0,
                                    inplace=True)

merged_df['Date'] = pd.to_datetime(merged_df['Date'])

weekly_time_series_df = merged_df.resample('W', on='Date').agg({
    'Calls Count': 'sum',
    'Deals Count': 'sum',
    'Conversion Rate': 'mean',
    'Calls per Deal': 'mean'
}).reset_index()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=weekly_time_series_df['Date'],
    y=weekly_time_series_df['Conversion Rate'],
    mode='lines+markers',
    name='Конверсия звонков в сделки (%)',
    line=dict(color='#08306b')
))

fig.add_trace(go.Scatter(
    x=weekly_time_series_df['Date'],
    y=weekly_time_series_df['Calls per Deal'],
    mode='lines+markers',
    name='Среднее кол-во звонков на сделку',
    line=dict(color='#60a7d2'),
    yaxis="y2"
))

fig.update_layout(
    title=dict(
        text='Анализ эффективности звонков',
        x=0.5,
        font=dict(size=24)
    ),
    yaxis_title='Конверсия звонков в сделки (%)',
    yaxis2=dict(
        title='Среднее кол-во звонков на сделку',
        overlaying='y',
        side='right'
    ),
    legend_title='Метрика',
    xaxis=dict(tickangle=45),
    template='plotly_white',
    legend=dict(
        x=1,
        y=1,
        xanchor='right',
        yanchor='bottom'
    )
)

fig.show()

*Обратная зависимость: Когда коэффициент конверсии высокий (например, март 2024 — 6%), среднее количество звонков на сделку низкое (~4). Когда конверсия падает (например, июль 2023 — 0%), количество звонков на сделку резко возрастает (до 10).*
*Это указывает на то, что в периоды низкой конверсии требуется больше звонков для закрытия сделки, что снижает эффективность работы.*

In [ ]:
merged_df[(merged_df['Date'] >= '2023-12-29') & (merged_df['Date'] <= '2024-01-10')]


,Date,Calls Count,Deals Count,Conversion Rate,Calls per Deal
180,2023-12-29,278,33.0,11.870504,8.424242
181,2023-12-30,98,19.0,19.387755,5.157895
182,2023-12-31,1,2.0,200.000000,0.500000
183,2024-01-01,5,59.0,1180.000000,0.084746
184,2024-01-02,202,46.0,22.772277,4.391304
185,2024-01-03,254,64.0,25.196850,3.968750
186,2024-01-04,308,59.0,19.155844,5.220339
187,2024-01-05,278,46.0,16.546763,6.043478
188,2024-01-06,107,36.0,33.644860,2.972222
189,2024-01-07,37,49.0,132.432432,0.755102


*Явный выброс на 2024-01-01 — конверсия 1180% при 5 звонках и 59 сделках. Это либо ошибка данных, либо нестандартная ситуация.*
*Сделки могли быть перенесены с предыдущих дней или могли закрыться сделки, накопленные за предыдущие дни.*

**Определение звязи между количеством звонков и сделок**

In [ ]:
fig2 = px.scatter(
    merged_df,
    x="Calls Count",
    y="Deals Count",
    title="Связь между количеством звонков и сделок",
    labels={
        "Calls Count": "Количество звонков",
        "Deals Count": "Количество сделок"
    },
    trendline="ols",
    color_discrete_sequence=["#60a7d2"],
    opacity=0.8,
)

fig2.data[1].line.color = "#6080b2"

fig2.update_layout(
    title_x=0.5,
    template='plotly_white',
    legend=dict(
        title="Пояснение",
        x=1,
        y=0.5,
        xanchor="left",
        yanchor="middle"
    )
)
fig2.update_traces(
    marker=dict(
        size=14,
        opacity=0.5
    )
)
fig2.show()

*График показывает, что с увеличением количества звонков увеличивается и количество сделок, это указывает на положительную корреляцию.*

## 2. Изучите распределение времени закрытия сделок и продолжительность
периода от создания до закрытия.


In [ ]:
deals_df['Deal Duration'] = (
    deals_df['Closing Date'] - deals_df['Created Time']
).dt.days.abs()

successful_deals_time = deals_df[
    (deals_df['Stage'] == 'Payment Done') & (deals_df['Deal Duration'] < 120)
]['Deal Duration'].dropna()

lost_deals_time = deals_df[
    (deals_df['Stage'] == 'Lost') & (deals_df['Deal Duration'] < 120)
]['Deal Duration'].dropna()

hist_data = [successful_deals_time, lost_deals_time]
group_labels = ['Успешные сделки', 'Потерянные сделки']

fig = ff.create_distplot(
    hist_data,
    group_labels,
    show_hist=True,
    show_rug=False,
    bin_size=5,
    colors=['#60a7d2', '#ff910d']
)

fig = ff.create_distplot(
    hist_data,
    group_labels,
    show_hist=True,
    show_rug=False,
    bin_size=5,
    colors=['#08306b', '#cfb09b']
)

fig.update_layout(
    title={
        'text': "Распределение времени до закрытия сделок",
        'x': 0.5,
        'font': {'size': 20}
    },
    xaxis_title="Время до закрытия (дни)",
    yaxis_title="Плотность распределения",
    template='plotly_white',
    legend_title="Тип сделок",
    bargap=0.05
)

for trace in fig.data:
    if trace.name == 'Потерянные сделки':
        trace.opacity = 0.6

fig.show()

*Большая часть сделок закрывается в первые дни.На графике виден резкий пик в районе 0-10 дней, что указывает на быструю обработку большинства сделок. Это касается как успешных, так и потерянных сделок.*
*Чем больше времени проходит, тем меньше вероятность закрытия сделки.*
*Потерянные сделки чаще закрываются быстрее. Успешные сделки тянутся дольше, но у них более равномерное распределение. Это может означать, что успешные сделки требуют больше времени на обработку.*

In [ ]:
deals_df['Closing Date'] = pd.to_datetime(deals_df['Closing Date'])
deals_df['Closing Month'] = deals_df['Closing Date'].dt.to_period('M')

deals_df['Month Start'] = deals_df['Closing Month'].dt.start_time

deals_df_filtered = deals_df[
    (deals_df['Month Start'] >= pd.Timestamp('2023-05-01')) &
    (deals_df['Month Start'] <= pd.Timestamp('2024-08-01'))
]

close_deals_per_month = deals_df_filtered.groupby('Month Start').agg(
    Close_Deals_Count=('Month Start', 'size'),
    Avg_Deal_Duration=('Deal Duration', 'mean')
).reset_index()


fig = go.Figure()

fig.add_trace(go.Bar(
    x=close_deals_per_month['Month Start'],
    y=close_deals_per_month['Close_Deals_Count'],
    name='Закрытые сделки',
    marker=dict(color='#85a8df')
))

fig.add_trace(go.Scatter(
    x=close_deals_per_month['Month Start'],
    y=close_deals_per_month['Avg_Deal_Duration'],
    mode='lines+markers',
    name='Средняя продолжительность сделки',
    line=dict(color='orange', width=2, dash='dash'),
    marker=dict(size=8, color='orange'),
    yaxis="y2"
))

fig.update_layout(
    title='Закрытие сделок и средняя продолжительность по месяцам',
    title_x=0.5,
    yaxis_title='Количество закрытых сделок',
    yaxis2=dict(
        title='Средняя продолжительность сделки (дней)',
        overlaying='y',
        side='right'
    ),
    template='plotly_white',
    showlegend=True,
    legend=dict(
        x=1,
        y=1.1,
        traceorder='normal',
        orientation='v',
        xanchor='left',
        yanchor='top'
    ),
    xaxis=dict(tickformat='%b %Y')
)

fig.show()

*С июля 2023 года наблюдается устойчивый рост количества закрытых сделок. Пик приходится на апрель 2024 года, после чего число сделок снижается. В первой половине периода (до ноября 2023) средняя продолжительность сделки увеличивается. Затем происходит спад в декабре 2023 – январе 2024. С февраля 2024 снова рост, достигая максимума в мае 2024, после чего наблюдается резкое снижение.*

*Рост количества сделок может быть связан с сезонностью или маркетинговыми активностями. Средняя продолжительность сделки не имеет четкой корреляции с количеством закрытых сделок. В периоды роста количества сделок продолжительность может как увеличиваться (август-ноябрь 2023, февраль-май 2024), так и снижаться (декабрь 2023 – январь 2024). Снижение количества сделок в июне 2024 может быть связано с сезонностью или изменением спроса.*

In [ ]:
colors = ['#08306b', '#2a4d8a', '#3e6aa4', '#6080b2', '#85a8df', '#a8d0f5',
          '#ff910d', '#ffb347', '#9c7648', '#b69272', '#cfb09b', '#dfd1ba']

fig = px.box(deals_df,
             x='Deal Duration',
             y='Stage',
             title='Длительность сделки в зависимости от стадии',
             color='Stage',
             color_discrete_sequence=colors)

fig.update_layout(
    xaxis_title='Длительность сделки (в днях)',
    yaxis_title='',
    template='plotly_white',
    title={
        'text': 'Длительность сделки в зависимости от стадии',
        'x': 0.5,
        'xanchor': 'center'
    },
    showlegend=False
)

fig.show()

*Большинство сделок на стадии  LOST имеют короткую длительность — в основном до 11 дней, но некоторые доходят до 300-350 дней. Это говорит о том, что некоторые сделки "зависают" на стадии потери, возможно, из-за отсутствия дальнейших действий.*
*Стадии, связанные с необходимостью звонков CALL DELAY, также показывают концентрацию в диапазоне до 30 дней, но есть редкие выбросы до 200-250 дней.*
*Средние стадии QUALIFIED, REGISTERED ON WEBINAR, WAITING FOR PAYMENT, NEED TO CALL - SALES показывают меньшее количество сделок, но длительность начинает увеличиваться.Это может быть связано с нерешительностью клиентов, сложностями в оплате или недостаточной мотивацией для продолжения.*
*WAITING FOR PAYMENT показывает, что некоторые сделки "зависают" на этой стадии дольше — до 105 дней, что может указывать на проблемы с процессом оплаты или нерешительность клиентов. Это может быть связано с тем, что клиенты, дошедшие до этой стадии, более мотивированы, но процесс обучения или финализации занимает время.*

*Улучшить процесс квалификации лидов на ранних стадиях, чтобы сократить количество "потерянных" сделок.*
*Оптимизировать этапы, связанные с ожиданием оплаты, возможно, внедрив более гибкие способы оплаты или напоминания.*
*Ускорить процесс на стадиях, связанных со звонками CALL DELAY, чтобы минимизировать задержки.*

**Анализ распределения закрытых сделок по стадиям**

In [ ]:
closed_deals = deals_df[deals_df['Closing Date'].notnull()]
stage_distribution = closed_deals['Stage'].value_counts().\
                      reset_index(name='Count')
stage_distribution.columns = ['Stage', 'Count']

colors = ['#85a8df', '#6080b2', '#08306b', '#b69272',
          '#a8d0f5', '#ff910d', '#ffb347', '#9c7648']

fig = px.pie(
    stage_distribution,
    names='Stage',
    values='Count',
    title='Распределение закрытых сделок по стадиям',
    color_discrete_sequence=colors,
    hole=0.4
)

fig.update_traces(
    textinfo='percent+label',
    textposition='outside',
    textfont_size=14,
    rotation=90,
    direction='clockwise',
    pull=[0.02] * len(stage_distribution),
    showlegend=False
)

fig.update_layout(
    template='plotly_white',
    title_x=0.5,
    height=800,
    title_y=0.95,
    title_font=dict(size=20),
    margin=dict(t=120, b=50, l=50, r=50),
    uniformtext_minsize=10,
    uniformtext_mode='hide'
)

fig.show()

*Огромная доля сделок (94,1%) теряется, что является ключевой проблемой воронки. Переход через воронку слабый, с минимальными процентами на стадиях до Payment Done. Стадии вроде Registered on Webinar и Need to Call - Sales имеют почти нулевые показатели, что указывает на неэффективность начального этапа.*

**Определение причин потери сделок**

In [ ]:
lost_deals = deals_df[deals_df['Stage'] == 'Lost']
lost_reason_distribution = lost_deals['Lost Reason'].value_counts().\
                            reset_index(name='Count')
lost_reason_distribution.columns = ['Lost Reason', 'Count']

fig = px.bar(
    lost_reason_distribution,
    y='Lost Reason',
    x='Count',
    title='Распределение потерянных сделок по причинам',
    color='Count',
    color_continuous_scale='Blues',
    orientation='h',
    text='Count'
)
fig.update_traces(
    textposition='outside',
    textfont_size=12,
    texttemplate='%{text}'
)

fig.update_layout(
    template='plotly_white',
    title_x=0.5,
    title_y=0.95,
    title_font=dict(size=20),
    xaxis_title="Количество сделок",
    yaxis_title="",
    showlegend=False,
    margin=dict(l=50, r=50, t=100, b=50),
    xaxis=dict(showticklabels=False),
    coloraxis_showscale=False
)

fig.show()

*Самая частая причина потерь – "Doesn't Answer" означает, что значительная часть потенциальных клиентов не выходит на связь.*
*"Changed Decision" – вторая по популярности причина говорит о том, что клиенты находят альтернативные варианты или теряют интерес.*
*"Non target" – значительная часть лидов не соответствует требованиям бизнеса, что указывает на возможные проблемы с квалификацией лидов*

*Нужна оптимизация процесса коммуникации, важно разработать стратегию повторного контакта с клиентами, которые не отвечают. Улучшить таргетинг, чтобы уменьшить количество нецелевых обращений. Отработать сценарии работы с клиентами, которые считают продукт дорогим или нуждаются в размышлении.*

# **Анализ эффективности кампаний**

## 1. Сравните эффективность различных кампаний
 с точки зрения генерации лидов и коэффициента конверсии.


**Сведение данных по кампаниям в таблицу для анализа основных показателей**

In [ ]:
deals_df_filtered = deals_df.dropna(subset=["Campaign"])

leads_per_campaign = deals_df_filtered.groupby("Campaign").size()
filtered_leads_per_campaign = leads_per_campaign[leads_per_campaign >= 30]

successful_deals_per_campaign = (
    deals_df_filtered[deals_df_filtered["Stage"].isin(["Payment Done"])]
    .groupby("Campaign")
    .size()
    .reindex(filtered_leads_per_campaign.index, fill_value=0)
)

conversion_rate_percent = (
    successful_deals_per_campaign / filtered_leads_per_campaign * 100).round(1)

campaign_data = pd.DataFrame({
    "Количество лидов": filtered_leads_per_campaign,
    "Успешные сделки": successful_deals_per_campaign,
    "Коэффициент конверсии (%)": conversion_rate_percent
}).reset_index()

campaign_data_filtered = campaign_data[campaign_data["Успешные сделки"] > 0]
campaign_data_sorted = campaign_data_filtered.sort_values(
    by=["Количество лидов", "Коэффициент конверсии (%)"],
    ascending=[False, False]
)

print(campaign_data_sorted.to_string(index=False))

                         Campaign  Количество лидов  Успешные сделки  Коэффициент конверсии (%)
performancemax_digitalmarkt_ru_DE              2573              112                        4.4
                youtube_shorts_DE              1596               53                        3.3
                12.07.2023wide_DE              1531               48                        3.1
                  02.07.23wide_DE               943               52                        5.5
         04.07.23recentlymoved_DE               734               31                        4.2
                    03.07.23women               595               31                        5.2
                           Dis_DE               567               30                        5.3
                   07.07.23LAL_DE               529               28                        5.3
        12.09.23interests_Uxui_DE               515               27                        5.2
           24.09.23retargeting_DE       

*Кампании brand_search_eng_DE, gen_analyst_DE, web2408_DE, karta_DE дают лучший коэффициент конверсии (>6%). Их стоит масштабировать.*
*Кампании с большим количеством лидов (performancemax_digitalmarkt_ru_DE, youtube_shorts_DE, 12.07.2023wide_DE) тоже могут быть эффективны при улучшении конверсии.*

*Кампании с низкой конверсией (менее 2%) – webinar1604, 08.04.24wide_webinar_DE, performancemax_eng_DE – возможно, нецелевые аудитории или проблемы с воронкой продаж.*


**Анализ крупных кампаний по успешности**

In [ ]:
campaigns_with_success = campaign_data_sorted["Campaign"]
campaigns_without_success = campaign_data[campaign_data["Успешные сделки"] == 0]["Campaign"]
sorted_campaigns_with_success = campaigns_with_success

common_layout = dict(
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=60, l=40, r=40, b=120),
    font=dict(family='Arial', size=12, color='#404040'),
    hoverlabel=dict(
        bgcolor='white',
        font_size=12,
        font_family='Arial'
    )
)

axis_style = dict(
    showgrid=True,
    gridcolor='#f0f0f0',
    linecolor='#d0d0d0',
    linewidth=1,
    zeroline=False,
    ticks='outside',
    tickcolor='#d0d0d0'
)

fig1 = go.Figure()

fig1.add_trace(go.Bar(
    x=sorted_campaigns_with_success,
    y=filtered_leads_per_campaign.loc[sorted_campaigns_with_success],
    name="Все сделки",
    marker_color='rgba(96,167,210,0.5)',
    hoverinfo='y+name',
    text=filtered_leads_per_campaign.loc[sorted_campaigns_with_success],
    textposition='inside',
    textfont=dict(size=10)
))

fig1.add_trace(go.Bar(
    x=sorted_campaigns_with_success,
    y=successful_deals_per_campaign.loc[sorted_campaigns_with_success],
    name="Успешные сделки",
    marker_color='rgba(8,48,107,0.9)',
    hoverinfo='y+name',
    text=successful_deals_per_campaign.loc[sorted_campaigns_with_success],
    textposition='inside',
    textfont=dict(size=10)
))

fig1.update_layout(
    **common_layout,
    title=dict(
        text="Кампании с успешными сделками (сделок >= 30)",
        x=0.5,
        xanchor='center',
        font=dict(size=16)
    ),
    xaxis=dict(
        **axis_style,
        title=None,
        tickangle=45,
        tickfont=dict(size=10),
        automargin=True
    ),
    yaxis=dict(
        **axis_style,
        title='Количество сделок'
    ),
    barmode='stack',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    hovermode='x unified',
    bargap=0.2
)

fig1.update_traces(
    texttemplate='%{text}',
    textposition='inside'
)

sorted_campaigns_without_success = campaigns_without_success.sort_values(
    key=lambda x: filtered_leads_per_campaign.loc[x], ascending=False
)

fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=sorted_campaigns_without_success,
    y=filtered_leads_per_campaign.loc[sorted_campaigns_without_success],
    name="Все сделки",
    marker_color='rgba(96,167,210,0.7)',
    hoverinfo='y+name',
    text=filtered_leads_per_campaign.loc[sorted_campaigns_without_success],
    textposition='inside',
    textfont=dict(size=10)
))

fig2.update_layout(
    **common_layout,
    title=dict(
        text="Кампании без успешных сделок (сделок >= 30)",
        x=0.5,
        xanchor='center',
        font=dict(size=16)
    ),
    xaxis=dict(
        **axis_style,
        title=None,
        tickangle=45,
        tickfont=dict(size=10),
        automargin=True
    ),
    yaxis=dict(
        **axis_style,
        title='Количество сделок'
    ),
    showlegend=False,
    hovermode='x unified',
    bargap=0.2
)

fig2.update_traces(
    texttemplate='%{text}',
    textposition='inside'
)

for fig in [fig1, fig2]:
    fig.update_layout(
        xaxis=dict(showline=True, mirror=True),
        yaxis=dict(showline=True, mirror=True)
    )

fig1.show()
fig2.show()

*Кампании с большим числом лидов, но очень низкой конверсией (<1%), например, 08.04.24wide_webinar_DE – возможно, лучше перераспределить бюджет на более эффективные источники.*

*Повысить конверсию лидеров (performancemax_digitalmarkt_ru_DE, youtube_shorts_DE) → тестировать новые подходы в работе с клиентами.*
*Перенаправить бюджет с неэффективных кампаний (08.04.24wide_webinar_DE, performancemax_eng_DE) на те, где уже есть высокая конверсия.*

**Создание слайдера для анализа кампаний по количеству лидов и конверсии**

In [ ]:
def plot_campaigns(num_campaigns=20):
    filtered_data = (
        campaign_data_sorted
        .sort_values("Количество лидов", ascending=False)
        .head(num_campaigns)
    )

    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=filtered_data['Campaign'],
        y=filtered_data['Количество лидов'],
        name='Лиды',
        marker_color='#60a7d2',
    ))

    fig.add_trace(go.Scatter(
        x=filtered_data['Campaign'],
        y=filtered_data['Коэффициент конверсии (%)'],
        mode='lines+markers+text',
        name='Конверсия (%)',
        line=dict(color='#08306b', width=3, shape='linear'),
        marker=dict(size=8, color='#08306b'),
        text=[f"{x}%" for x in filtered_data['Коэффициент конверсии (%)']],
        textposition='top center',
        yaxis='y2',
        connectgaps=True
    ))

    fig.update_layout(
        title=dict(
            text=f'Топ-{num_campaigns} кампаний по количеству лидов и конверсии',
            x=0.5,
            y=0.95,
            font=dict(size=24)
        ),
        xaxis=dict(
            title='',
            tickangle=45,
            type='category',
            categoryorder='array',
            categoryarray=filtered_data['Campaign'],
            tickmode='array',
            tickvals=filtered_data['Campaign']
        ),
        yaxis=dict(
            title='Лиды',
            range=[0, filtered_data['Количество лидов'].max() * 1.05],
            showgrid=True,
            gridcolor='lightgrey'
        ),
        yaxis2=dict(
            title='Конверсия (%)',
            overlaying='y',
            side='right',
            range=[0, 40],
            showgrid=False
        ),
        template='plotly_white',
        margin=dict(t=100, b=150, l=50, r=100),
        height=800,
        width=1600,
        hovermode='x unified'
    )

    fig.show(config={'responsive': False})

max_campaigns = len(campaign_data_sorted)
interact(
    plot_campaigns,
    num_campaigns=IntSlider(
        min=1,
        max=max_campaigns,
        value=10,
        description='Количество кампаний:',
        style={'description_width': '150px'},
        layout={'width': '100%'}
    )
)

*Высокий трафик не всегда означает высокую конверсию – крупнейшие кампании имеют конверсию 4-6%.*
*Некоторые кампании (правее на графике) дают более высокую конверсию, например, 37.5% у одной из них, но они имеют мало лидов, что делает такие результаты менее информативными. Высокая конверсия при небольшом числе клиентов может быть случайной и не отражать реальную эффективность кампании.*
*Оптимизация крупных кампаний может принести значительный эффект – даже +1% конверсии у кампании с 2500 лидами даст 25 дополнительных клиентов. *

## 2. Оцените эффективность различных маркетинговых источников (Source)
 в генерировании качественных лидов.

In [ ]:
successful_deals_per_source = deals_df[
    deals_df["Stage"].isin(["Payment Done"])
].groupby("Source").size()

leads_per_source = deals_df.groupby("Source").size()

conversion_rate_per_source = (
    successful_deals_per_source.div(leads_per_source, fill_value=0)
    * 100
).round(1)

source_data = pd.DataFrame({
    "Количество лидов": leads_per_source,
    "Успешные сделки": successful_deals_per_source,
    "Конверсия (%)": conversion_rate_per_source
})

source_data_sorted = source_data.sort_values(
    by="Количество лидов",
    ascending=False
)

print(source_data_sorted.to_string())

                Количество лидов  Успешные сделки  Конверсия (%)
Source                                                          
Facebook Ads                4729            202.0            4.3
Google Ads                  4114            172.0            4.2
Tiktok Ads                  2003             56.0            2.8
SMM                         1669             91.0            5.5
Youtube Ads                 1618             53.0            3.3
Organic                     1496            145.0            9.7
CRM                         1455             24.0            1.6
Bloggers                    1074             39.0            3.6
Telegram posts               993             40.0            4.0
Webinar                      303             26.0            8.6
Partnership                  201              4.0            2.0
Offline                        2              NaN            0.0


* *Organic (9.7%) – органический трафик показывает наилучшую конверсию,* *несмотря на сравнительно небольшое количество лидов(1496).* *Это может говорить о более целевой и заинтересованной аудитории.*
* *Webinar (8.6%) – вебинары также демонстрируют высокий уровень конверсии, что свидетельствует о качестве лида и более высокой вероятности сделки.*
* *CRM (1.6%) – несмотря на 1455 лидов, этот источник имеет самую низкую конверсию среди значимых каналов. Возможно, требуется улучшение стратегии работы с базой.*
* *Partnership (2.0%) и Tiktok Ads (2.8%) также имеют низкую эффективность в плане сделок.*

**Анализ источников по эффективности**

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=source_data_sorted.index,
    y=source_data_sorted['Успешные сделки'],
    name='Успешные сделки',
    marker_color='#08306b',
    text=source_data_sorted['Успешные сделки'],
    textposition='outside',
    texttemplate='%{y:,}',
    offsetgroup=1
))

fig.add_trace(go.Bar(
    x=source_data_sorted.index,
    y=source_data_sorted['Количество лидов'],
    name='Количество лидов',
    marker=dict(
        color='rgba(96, 167, 210, 0.5)',
        line=dict(color='#60a7d2', width=1)
    ),
    offsetgroup=1
))

fig.add_trace(go.Scatter(
    x=source_data_sorted.index,
    y=source_data_sorted['Конверсия (%)'],
    mode='lines+markers+text',
    name='Конверсия (%)',
    line=dict(color='#08306b', width=3),
    marker=dict(size=8, color='#08306b'),
    text=[f"{x}%" for x in source_data_sorted['Конверсия (%)']],
    textposition='top center',
    yaxis='y2'
))

fig.update_layout(
    title=dict(
        text='Эффективность источников: Лиды, Сделки и Конверсия',
        x=0.5,
        y=0.95,
        font=dict(size=24)
    ),
    xaxis=dict(
        title='',
        tickangle=45,
        type='category',
        categoryorder='total descending'
    ),
    yaxis=dict(
        title='Количество (лиды и сделки)',
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis2=dict(
        title='Конверсия (%)',
        overlaying='y',
        side='right',
        range=[0, source_data_sorted['Конверсия (%)'].max() * 1.5],
        showgrid=False
    ),
    template='plotly_white',
    margin=dict(t=100, b=150, l=50, r=100),
    hovermode='x unified',
    legend=dict(x=1.1, y=1),
    barmode='group'
)

fig.show()

*Оптимальные источники: Organic, Webinar, SMM – дают высокую конверсию. Facebook Ads и Google Ads – дают наибольшее количество лидов, но требуют улучшения конверсии. CRM и Tiktok Ads – низкая конверсия. Маленькие выборки с высокой конверсией (Webinar, Partnership) могут быть неинформативными для масштабирования.*

*Нужно: оптимизировать Facebook Ads и Google Ads (например, через ретаргетинг или изменение воронки продаж).*
*Увеличить трафик с Organic и Webinar, так как их качество лидов выше.*
*Пересмотреть стратегию работы с CRM и Tiktok Ads – проверить, насколько лиды из этих источников соответствуют целевой аудитории. *

**Анализ качества сделок по источникам**

In [ ]:
heatmap_data = deals_df.groupby(['Source', 'Quality']).size().\
                                  reset_index(name='Count')

heatmap_pivot = heatmap_data.pivot(index='Quality', columns='Source',\
                                   values='Count').fillna(0)

total_count = heatmap_pivot.sum().sum()
heatmap_percent = ((heatmap_pivot / total_count) * 100).round(1)

fig_heatmap = px.imshow(
    heatmap_percent,
    labels={'x': 'Источник', 'y': 'Качество', 'color': 'Процент сделок'},
    title='Качество сделок по источникам (в %)',
    color_continuous_scale='Blues',
    text_auto=True
)

fig_heatmap.update_layout(
    title_x=0.5,
    yaxis_title="Качество",
    width=1200,
    height=600,
    coloraxis_showscale=False,
    xaxis=dict(tickangle=45)
)

fig_heatmap.show()

*Facebook Ads (8% D, 8.7% E) и Google Ads (7.7% D, 9ю5% E) генерируют значительную долю некачественных лидов.*
*Почти все источники имеют очень низкую долю сделок класса A (максимум 0.6%). Это указывает на сложность привлечения качественных клиентов.Organic (9.8% конверсия на предыдущем графике) показывает относительно низкую долю низкокачественных сделок. Tiktok и SMM также демонстрируют баланс качества.*

*Стоит проверить фильтрацию лидов на входе, чтобы исключить нерелевантные заявки. Возможно, стоит перераспределить бюджет в пользу источников, которые дают лучшую конверсию и качество.*

# **Анализ эффективности работы отдела продаж:**


## 1. Оцените эффективность отдельных владельцев сделок и рекламных кампаний
с точки зрения количества обработанных сделок, коэффициента конверсии и
общей суммы продаж.

In [ ]:
deals_df_clean = deals_df.dropna(
    subset=["Deal Owner Name"]
)

def analyze_owner_performance(df, sort_by="Общая сумма продаж (€)"):
    successful_deals = (
        df[df["Stage"] == "Payment Done"]
        .groupby("Deal Owner Name")
        .size()
    )
    total_leads = df.groupby("Deal Owner Name").size()
    conversion_rate = (successful_deals / total_leads).fillna(0)

    total_sales = (
        df[df["Stage"] == "Payment Done"]
        .groupby("Deal Owner Name")["Offer Total Amount (€)"]
        .sum()
    )

    performance_df = pd.DataFrame({
        "Обработано сделок": total_leads,
        "Успешные сделки": successful_deals.fillna(0),
        "Конверсия (%)": (conversion_rate * 100).round(2),
        "Общая сумма продаж (€)": total_sales
    }).fillna(0)

    return performance_df.sort_values(by=sort_by, ascending=False)


owner_performance = analyze_owner_performance(deals_df_clean)

print(
    "Владельцы сделок по количеству обработанных, успешных сделок, "
    "конверсии и общей выручке:"
)
print(
    owner_performance.to_string(
        formatters={
            "Общая сумма продаж (€)": "{:,.0f} €".format,
            "Конверсия (%)": "{:.1f}%".format
        }
    )
)

Владельцы сделок по количеству обработанных, успешных сделок, конверсии и общей выручке:
                 Обработано сделок  Успешные сделки Конверсия (%) Общая сумма продаж (€)
Deal Owner Name                                                                         
Charlie Davis                 2788            148.0          5.3%            1,066,600 €
Ulysses Adams                 2057            140.0          6.8%            1,009,400 €
Julia Nelson                  2061             91.0          4.4%              718,700 €
Paula Underwood               1749             91.0          5.2%              672,000 €
Oliver Taylor                  160             50.0         31.2%              524,500 €
Quincy Vincent                1782             65.0          3.6%              465,500 €
Victor Barnes                 1182             44.0          3.7%              356,000 €
Ben Hall                      1287             46.0          3.6%              345,500 €
Nina Scott           

*Charlie Davis (2788 сделок) и Ulysses Adams (2057 сделок) обработали наибольшее количество сделок. Это говорит о высокой активности и вовлеченности в процесс продаж.Наименьшее количество сделок обработали Wendy Clark (1 сделка) и Zachary Foster (1 сделка). Их вклад в общий объем продаж минимален.*
*Высокий объём обработанных сделок не всегда гарантирует лучшую конверсию, но даёт высокий общий доход*.

*Oliver Taylor — абсолютный лидер по конверсии, но у него мало обработанных сделок. Kevin Parker и John Doe демонстрируют хорошие проценты, но их вклад в общую выручку низкий.*

 *Оптимизировать стратегию для сильных продавцов (Cara, Charlie, Ulysses), продолжать поддерживать их активность, но обратить внимание на повышение конверсии. Возможно, стоит оптимизировать процессы или обучить их работе с более сложными сделками. Обучить слабых исполнителей (Diana, Ian, Mason) или проверить качество их лидов.*

**Анализ эффективности отдельных владельцев сделок с точки зрения количества обработанных и успешных сделок**

In [ ]:
filtered_data = owner_performance[owner_performance['Обработано сделок'] > 0]

active_owners = filtered_data[filtered_data['Успешные сделки'] > 0]
zero_owners = filtered_data[filtered_data['Успешные сделки'] == 0]

active_sorted = active_owners.sort_values(by='Обработано сделок', ascending=False)
zero_sorted = zero_owners.sort_values(by='Обработано сделок', ascending=False)

def create_horizontal_bar_chart(df, title):
    fig = go.Figure()

    fig.add_trace(go.Bar(
        y=df.index,
        x=df['Обработано сделок'],
        name='Обработано сделок',
        orientation='h',
        marker=dict(
            color='rgba(8, 48, 107, 0.9)',
            line=dict(color='#89CFF0', width=1)
        ),
        width=0.6,
        text=df['Обработано сделок'],
        textposition='outside',
        texttemplate='%{x:,}',
        textfont=dict(
            color='darkblue',
            size=12,
            family='Arial'
        )
    ))

    fig.add_trace(go.Bar(
        y=df.index,
        x=df['Успешные сделки'],
        name='Успешные сделки',
        orientation='h',
        marker=dict(
            color='rgba(96, 167, 210, 0.4)',
            line=dict(color='#0066CC', width=1.5)
        ),
        width=0.8,
        text=df['Успешные сделки'],
        textposition='outside',
        texttemplate='%{x:,}',
        textfont=dict(
            color='white',
            size=12,
            family='Arial'
        )
    ))

    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,
            y=0.95,
            xanchor='center',
            font=dict(size=24, family='Arial')
        ),
        xaxis=dict(
            title=None,
            showgrid=False,
            showticklabels=False,
            range=[0, df['Обработано сделок'].max() * 1.1]
        ),
        yaxis=dict(
            title='',
            showgrid=False,
            autorange='reversed'
        ),
        plot_bgcolor='white',
        barmode='overlay',
        height=600,
        width=1400,
        margin=dict(l=200, r=200),
        legend=dict(
            x=1,
            y=0.05,
            xanchor='right',
            yanchor='bottom'
        )
    )
    return fig

if not active_sorted.empty:
    fig1 = create_horizontal_bar_chart(active_sorted, 'Владельцы с успешными сделками')
    fig1.show()

if not zero_sorted.empty:
    fig2 = create_horizontal_bar_chart(zero_sorted, 'Владельцы без успешных сделок')
    fig2.show()

*Лидеры по количеству обработанных сделок:*
*Charlie Davis: 2,788 обработанных сделок, из них 148 успешных.*
*Julia Nelson: 2,061 обработанных сделок, из них 91 успешная.*
*Ulysses Adams: 2,057 обработанных сделок, из них 65 успешных.*
*У большинства владельцев доля успешных сделок относительно мала по сравнению с общим количеством обработанных сделок. Это может указывать на то, что они берут на себя слишком много сделок, жертвуя качеством ради количества.*

*Второй график выделяет владельцев, у которых нет ни одной успешной сделки, несмотря на разное количество обработанных сделок (от 1 до 341). Это может быть связано с их неэффективной стратегией, отсутствием опыта или неблагоприятными условиями для сделок.*

**Определение доли сделок по стадиям и по менеджерам**

In [ ]:
deals_df_clean = deals_df_clean.dropna(subset=["Stage", "Deal Owner Name"])

stage_distribution = deals_df_clean.groupby(
    ["Deal Owner Name", "Stage"]
).size().unstack(fill_value=0)

stage_distribution_pct = stage_distribution.div(
    stage_distribution.sum(axis=1), axis=0
) * 100

fig = px.imshow(
    stage_distribution_pct.T.values,
    labels={
        "x": "Менеджер",
        "y": "Стадия сделки",
        "color": "Доля сделок (%)"
    },
    x=stage_distribution_pct.index,
    y=stage_distribution_pct.columns,
    color_continuous_scale="Blues",
    text_auto=".1f"
)

fig.update_traces(
    text=stage_distribution_pct.T.applymap(
        lambda x: "" if x == 0 else f"{x:.1f}"),
    texttemplate='%{text}',
    textfont=dict(size=12)
)

fig.update_layout(
    title_text="Доля сделок по менеджерам и стадиям (%)",
    title_x=0.5,
    xaxis_title="",
    yaxis_title="",
    xaxis=dict(tickangle=-45, title_standoff=10),
    yaxis=dict(title_standoff=10),
    coloraxis_colorbar_title="%",
    width=1400,
    height=800,
    coloraxis_showscale=False
)

fig.show()

К*атегория "Lost" имеет наибольшие значения, достигающие 100% у нескольких людей, что может означать высокий процент потерь на этом этапе процесса. Высокие значения в категории "Lost" могут сигнализировать о проблеме на определенном этапе процесса.Категория "Registered on Webinar" имеет умеренные значения (от 10% до 75%), но они варьируются среди разных людей.*
*"Registered on Webinar" показывает различия между пользователями, что может быть полезно для анализа эффективности регистрации.*
*Отдельные категории, такие как "Payment Done" и "Qualified", имеют низкие значения, что свидетельствует о проблемах с конверсией. Большинство других категорий имеют низкие значения (много нулей), что может указывать на редкие события.*

**Анализ конверсии и выручки по владельцам**

In [ ]:
owner_performance_filtered = owner_performance[
    owner_performance["Общая сумма продаж (€)"] > 0
]

owner_performance_sorted = owner_performance_filtered.sort_values(
    by="Общая сумма продаж (€)", ascending=False
)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=owner_performance_sorted.index,
    y=owner_performance_sorted["Общая сумма продаж (€)"],
    name="Выручка (€)",
    marker_color='#85a8df',
    text=owner_performance_sorted["Общая сумма продаж (€)"],
    texttemplate='%{y:,.0f}',
    textposition='outside'
))

fig.add_trace(go.Scatter(
    x=owner_performance_sorted.index,
    y=owner_performance_sorted["Конверсия (%)"],
    name="Конверсия (%)",
    line=dict(color='#08306b', width=3),
    yaxis='y2',
    mode='lines+markers',
    marker=dict(size=10, color='#08306b')
))

fig.update_layout(
    title=dict(
        text="Конверсия и выручка по владельцам",
        x=0.5,
        font=dict(size=24)
    ),
    xaxis=dict(
        title="",
        tickangle=45,
        type='category',
        categoryorder='total descending'
    ),
    yaxis=dict(
        title="Выручка (€)",
        showgrid=False,
        range=[0, owner_performance_sorted["Общая сумма продаж (€)"].max() * 1.1]
    ),
    yaxis2=dict(
        title="Конверсия (%)",
        overlaying='y',
        side='right',
        range=[0, 35],
        showgrid=False
    ),
    template="plotly_white",
    legend=dict(
        x=1.05,
        y=1,
        bgcolor='rgba(255,255,255,0.5)'
    ),
    margin=dict(t=100, b=150, l=50, r=100),

)

fig.show()

*Максимальная выручка у Cara Iverson, Charlie Davis и Ulysses Adams. Самая высокая конверсия у Oliver Taylor, но его выручка низкая. У владельцев с высокой выручкой конверсия в среднем невысокая, можно сделать вывод, что высокая конверсия не всегда коррелирует с высокой выручкой. Низкая выручка и почти нулевая конверсия у Mason Roberts, George King и Diana Evans.*
*Оптимальная стратегия — баланс между объемом сделок и качеством конверсии.*

**Анализ длительности взаимодействия с клиентами, количества клиентов по месяцам**


In [ ]:
deals_df['Created Time'] = pd.to_datetime(deals_df['Created Time'])
deals_df['Created Time'] = pd.to_datetime(deals_df['Created Time'])

deals_df_filtered = deals_df.copy()
deals_df_filtered['Month'] = deals_df_filtered['Created Time'].dt.to_period(
    'M'
).astype(str)

clients_per_consultant = deals_df_filtered.groupby(
    ['Month', 'Deal Owner Name']
).size().reset_index(name='Client Count')

pivot_data = clients_per_consultant.pivot_table(
    index='Deal Owner Name',
    columns='Month',
    values='Client Count',
    fill_value=0
)
pivot_data['Total'] = pivot_data.sum(axis=1)
pivot_data = pivot_data.sort_values('Total', ascending=False)
pivot_data_for_heatmap = pivot_data.drop(columns='Total')

fig_heatmap = px.imshow(
    pivot_data_for_heatmap,
    labels=dict(x="Месяц", y="Консультант", color="Количество клиентов"),
    title="Тепловая карта количества клиентов у консультантов по месяцам",
    color_continuous_scale='Blues',
    aspect="auto",
    text_auto=True
)
fig_heatmap.update_traces(
    text=pivot_data_for_heatmap.applymap(lambda x: "" if x == 0 else str(x)),
    texttemplate='%{text}',
    textfont=dict(size=12)
)

fig_heatmap.update_layout(
    title_x=0.5,
    width=1200,
    height=800,
    template='plotly_white',
    margin=dict(l=50, r=50, t=100, b=50),
    coloraxis_showscale=False
)

fig_heatmap.show()

*Консультанты, такие как Cara Iverson, Charlie Davis, Ulysses Adams и Ben Hall, являются лидерами, привлекая сотни клиентов в месяц. Они, вероятно, используют эффективные стратегии или работают с крупными проектами.*
*Wendy Clark, Xander Dean, Zachary Foster и Alice Johnson, Bob Brawn, Sam Young, George King, Mason Roberts практически не имеют активности, что может указывать на их уход или низкую эффективность.*
*Rachel White и Eva Kent демонстрируют устойчивый рост, особенно в последние месяцы. Их стратегии могут быть успешными и заслуживают внимания.*
*Консультанты с минимальной активностью (Yara Edwards, Amy Green) могут нуждаться в поддержке или пересмотре их подхода.*

*Сезонные пики:*
*Январь 2024 года по апрель 2024 года — самые активные месяцы, что может быть связано с сезонным спросом (например, начало года).*
*Май 2024 года показывает снижение активности у большинства консультантов, возможно из-за окончания проектов или сезонного спада.*

**Анализ среднего времени продолжительности откликов по менеджерам**

In [ ]:
owner_sla_avg = deals_df.groupby('Deal Owner Name', as_index=False)['SLA Seconds']\
                                  .mean()
owner_sla_avg['SLA Hours'] = owner_sla_avg['SLA Seconds'] / 3600
owner_sla_avg['Color'] = owner_sla_avg['SLA Hours'].apply(
    lambda x: 'red' if x > owner_sla_avg['SLA Hours'].quantile(0.75) else 'blue'
)
avg_sla = owner_sla_avg['SLA Hours'].mean()

fig = px.bar(
    owner_sla_avg,
    x='Deal Owner Name',
    y='SLA Hours',
    title='Среднее время отклика по владельцам сделок (часы)',
    color='Color',
    color_discrete_map={'red': '#ffb347', 'blue': '#a8d0f5'}
)

fig.add_hline(
    y=avg_sla,
    line_dash='dash',
    line_color='red',
    line_width=0.5,
    annotation_text="Среднее время отклика",
    annotation_position="top left"
)

fig.update_layout(
    xaxis_title='',
    yaxis_title='Среднее время отклика (часы)',
    xaxis_tickangle=45,
    title_x=0.5,
    title_y=0.95,
    title_xanchor='center',
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=False
)

fig.show()

*Большинство владельцев сделок имеют среднее время отклика ниже 40 часов.Самые долгие отклики  у Kevin Parker, Mason Roberts, Rachel White, Victor Barnes, Xander Dean – превышают 60 часов.*
*Долгое время отклика может негативно сказываться на конверсии и закрытии сделок.*
*Владельцам с высоким временем отклика стоит пересмотреть приоритеты в работе с клиентами.*

**Анализ по менеджерам: влияние длительности звонка на конверсию**

In [ ]:
calls_df_cleaned = calls_df.dropna(subset=['Call Duration (in seconds)'])

calls_agg = calls_df_cleaned.groupby('Call Owner Name', as_index=False).agg(
    Средняя_длительность_звонка=('Call Duration (in seconds)', lambda x: x.\
                                 mean() / 60))
calls_agg['Средняя_длительность_звонка'] = calls_agg[
    'Средняя_длительность_звонка'].round(2)

owner_performance = owner_performance.reset_index()

owner_performance_with_calls = owner_performance.merge(
    calls_agg,
    left_on='Deal Owner Name',
    right_on='Call Owner Name',
    how='left'
)

owner_performance_with_calls = owner_performance_with_calls.drop(
    columns=['Call Owner Name'])

owner_performance_with_calls['Средняя_длительность_звонка'] =\
 owner_performance_with_calls['Средняя_длительность_звонка'].fillna(0)

print(owner_performance_with_calls.to_string())

    index  Deal Owner Name  Обработано сделок  Успешные сделки  Конверсия (%)  Общая сумма продаж (€)  Средняя_длительность_звонка
0       0    Charlie Davis               2788            148.0           5.31               1066600.0                         3.75
1       1    Ulysses Adams               2057            140.0           6.81               1009400.0                         3.43
2       2     Julia Nelson               2061             91.0           4.42                718700.0                         3.47
3       3  Paula Underwood               1749             91.0           5.20                672000.0                         3.33
4       4    Oliver Taylor                160             50.0          31.25                524500.0                         1.73
5       5   Quincy Vincent               1782             65.0           3.65                465500.0                         4.67
6       6    Victor Barnes               1182             44.0           3.72      

*В целом, не наблюдается явной корреляции между длительностью звонков и количеством успешных сделок. Например, Charlie Davis, с длительностью звонка 3.75 минуты, имеет 148 успешных сделок, тогда как Victor Barnes, с длительностью 2.44 минуты, имеет только 44 успешные сделки.*
*С другой стороны, есть такие примеры как Oliver Taylor, который имеет крайне короткую длительность звонка (1.73 минуты), но его конверсия составляет 31.25%, что довольно высоко.Длительность звонков не всегда напрямую влияет на успешность сделок, и существует множество примеров, где краткие звонки оказываются столь же эффективными (или более эффективными), чем более длинные.*
*Высокая конверсия может быть достигнута как при коротких звонках, так и при более продолжительных, что указывает на важность других факторов, таких как качество взаимодействия, подход к клиенту и навыки продаж*.
*Некоторые агенты, которые проводят длинные звонки без успешных сделок, могут требовать пересмотра стратегии общения с клиентами.*

**Анализ влияния длительности звонка на конверсию**

In [ ]:
colors = ['#08306b', '#3e6aa4', '#6080b2', '#85a8df', '#a8d0f5', '#48c9f0',
    '#ff3a00', '#ffb347', '#ff910d', '#9c7648', '#b69272', '#cfb09b', '#dfd1ba']

fig = px.scatter(
    owner_performance_with_calls,
    x='Средняя_длительность_звонка',
    y='Конверсия (%)',
    size='Обработано сделок',
    color='Deal Owner Name',
    title='Влияние длительности звонков на конверсию',
    labels={'Avg_Call_Duration': 'Средняя длительность звонков (минуты)', 'Конверсия (%)': 'Конверсия (%)'},
    hover_data=['Общая сумма продаж (€)'],
    size_max=30,
    color_discrete_sequence=colors
)

fig.update_layout(
    xaxis_title='Средняя длительность звонков (минуты)',
    yaxis_title='Конверсия (%)',
    title_x=0.5,
    template='plotly_white',
    showlegend=True,
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='Менеджеры'
)

fig.show()

*Конверсия максимальна у менеджера Oliver Taylor, при этом средняя длительность звонков составляет 1.73 мин. Средняя длительность звонков по менеджерам составляет от 2 до 4.67 мин при этом конверсия распределена в пределах 3-6%.*
Менеджеры с большим количеством звонков (например, Charlie Davis, Ulysses Adams) показывают низкую конверсию, что может быть связано с перегрузкой. Короткие звонки могут быть более эффективными, если лиды хорошо квалифицированы. Длинные звонки (например, у Ben Hall) не гарантируют высокую конверсию.
*Необходимо проанализировать методы работы наиболее эффективных менеджеров и внедрить их в общую практику.*

**Анализ эффективности отдельных владельцев сделок с точки конверсии и выручки**

In [ ]:
deals_df["Created Time"] = pd.to_datetime(deals_df["Created Time"], errors="raise")
deals_df["Closing Date"] = pd.to_datetime(deals_df["Closing Date"], errors="raise")

successful_deals = deals_df[deals_df["Stage"] == "Payment Done"]

deal_owner_detailed_analysis = deals_df.groupby("Deal Owner Name").agg(
    Total_Deals=("Stage", "count"),
    Converted_Deals=("Stage", lambda x: (x == "Payment Done").sum()),
    Conversion_Rate=(
        "Stage",
        lambda x: (x == "Payment Done").sum() / len(x) * 100 if len(x) > 0 else 0,
    ),
    Total_Sales=("Initial Amount Paid (€)", "sum"),
    Avg_Deal_Amount=(
        "Initial Amount Paid (€)",
        lambda x: x[x > 0].mean()
    ),
    Avg_Time_to_Close=(
        "Closing Date",
        lambda x: (x - deals_df.loc[x.index, "Created Time"]).dt.days.mean(),
    ),
).reset_index()

campaign_detailed_analysis = deals_df.groupby("Campaign").agg(
    Total_Deals=("Stage", "count"),
    Converted_Deals=("Stage", lambda x: (x == "Payment Done").sum()),
    Conversion_Rate=(
        "Stage",
        lambda x: (x == "Payment Done").sum() / len(x) * 100 if len(x) > 0 else 0,
    ),
    Total_Sales=("Initial Amount Paid (€)", "sum"),
    Avg_Deal_Amount=(
        "Initial Amount Paid (€)",
        lambda x: x[x > 0].mean()
    ),
    Avg_Time_to_Close=(
        "Closing Date",
        lambda x: (x - deals_df.loc[x.index, "Created Time"]).dt.days.mean(),
    ),
).reset_index()

stage_distribution_owner = deals_df.pivot_table(
    index="Deal Owner Name",
    columns="Stage",
    values="Id",
    aggfunc="count",
    fill_value=0
).reset_index()

stage_distribution_campaign = deals_df.pivot_table(
    index="Campaign",
    columns="Stage",
    values="Id",
    aggfunc="count",
    fill_value=0
).reset_index()

In [ ]:
colors = ['#08306b', '#3e6aa4', '#6080b2', '#85a8df', '#a8d0f5', '#48c9f0',
    '#ff3a00', '#ffb347', '#ff910d', '#9c7648', '#b69272', '#cfb09b', '#dfd1ba']

fig = px.bar(
    stage_distribution_owner,
    x='Deal Owner Name',
    y=stage_distribution_owner.columns[1:],
    title="Распределение сделок по стадиям и владельцам",
    labels={'value': 'Количество сделок', 'variable': 'Стадия',\
            'Deal Owner Name': 'Владелец сделки'},
    color_discrete_sequence=colors,
    barmode='stack'
)

fig.update_layout(
    xaxis={'categoryorder': 'total descending'},
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()

*Неравномерное распределение нагрузки:*
*Менеджеры, такие как Charlie Davis и Julia Nelson, обрабатывают значительно больше сделок, чем другие. Это может приводить к перегрузке и задержкам , что снижает эффективность работы.*
*Высокий уровень потерь:*
*У многих менеджеров значительная доля сделок находится на стадии LOST, особенно у тех, у кого меньше общего количества сделок. Потому есть необходимость улучшения квалификации лидов на ранних стадиях.*
*Низкая конверсия на поздних стадиях:*
 *Стадии, связанные с завершением сделок (PAYMENT DONE), встречаются редко, особенно у менеджеров с большим количеством сделок. Это может указывать на проблемы с конверсией на поздних этапах воронки продаж.*

In [ ]:
top_campaigns = campaign_detailed_analysis.nlargest(20, 'Total_Deals')['Campaign']

top_campaigns = top_campaigns.astype(str)
stage_distribution_campaign['Campaign'] = stage_distribution_campaign[\
                                          'Campaign'].astype(str)

missing_campaigns = [
    campaign for campaign in top_campaigns
    if campaign not in stage_distribution_campaign['Campaign'].values
]
print(missing_campaigns)

top_campaigns = campaign_detailed_analysis.nlargest(20, 'Total_Deals')['Campaign']

filtered_stage = (
    stage_distribution_campaign[stage_distribution_campaign['Campaign'].\
                                isin(top_campaigns)]
    .set_index('Campaign')
    .loc[top_campaigns]
    .reset_index()
)

fig = px.bar(
    filtered_stage,
    x='Campaign',
    y=filtered_stage.columns[2:],
    title="Распределение стадий по ключевым кампаниям",
    labels={'value': 'Количество сделок', 'variable': 'Стадия'},
    color_discrete_sequence=colors,
    category_orders={"Campaign": top_campaigns.tolist()}
)

fig.update_layout(
    barmode='stack',
    title_x=0.5,
    xaxis_title=None,
    yaxis_title="Количество сделок",
    legend_title="Стадия",
    hovermode='x unified',
    plot_bgcolor='white',
    height=600,
    margin=dict(r=50),
    legend=dict(
        orientation="v",
        x=1,
        y=0.5,
        xanchor="left",
        yanchor="middle"
    )
)

fig.update_xaxes(tickangle=45)
fig.update_traces(hovertemplate='%{y} сделок')

fig.show()

[]


*Кампания performancemax_digitalmarkt_ru_DE значительно опережает остальные, имея более 2000 сделок. youtube_shorts_DE и 12.07.2023wide_DE также входят в топ-3, но с меньшим отрывом.Большая часть сделок находится в статусе "Lost". Это может говорить о проблемах с качеством лидов или недостаточной обработке заявок.*
*Минимальное количество сделок дошло до этапа "Payment Done" (оплата завершена), что может указывать на проблемы с конверсией в оплату.В некоторых кампаниях (например, webinar1604, 08.04.24wide_webinar_DE) больше доля "Registered on Webinar" и "Waiting for Payment", что указывает на их ориентацию на вебинары, но слабо проработанную конверсию в оплату.*
*webinar1604, 08.04.24wide_webinar_DE и blog2_DE – здесь высокая доля "Registered on Webinar", но низкая доля "Payment Done". Это указывает на низкую конверсию из регистрации в оплату*.
*performancemax_wide_PL и discovery_DE – малое количество сделок, много потерянных лидов. Возможно, неэффективное таргетирование или плохая проработка лида.*

**Анализ количества успешных сделок и средней суммы сделки**

In [ ]:
top_n = 20
top_campaigns = campaign_detailed_analysis.nlargest(top_n, 'Converted_Deals')
scale_factor = np.max(top_campaigns['Converted_Deals']) / np.max(\
                top_campaigns['Avg_Deal_Amount'])

fig = go.Figure()

fig.add_trace(go.Bar(
    x=top_campaigns['Campaign'],
    y=top_campaigns['Converted_Deals'],
    name='Количество успешных сделок',
    marker_color='#08306b',
    text=top_campaigns['Converted_Deals'],
    textposition='inside',
    insidetextanchor='middle',
    textangle=270
))

fig.add_trace(go.Bar(
    x=top_campaigns['Campaign'],
    y=top_campaigns['Avg_Deal_Amount'] * scale_factor,
    name='Средняя сумма сделки',
    marker_color='#cfb09b',
    text=top_campaigns['Avg_Deal_Amount'].round(0),
    textposition='inside',
    insidetextanchor='middle',
    textangle=270
))

fig.update_layout(
    title="Количество успешных сделок и средняя сумма сделки по кампаниям",
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white',
    yaxis=dict(
        showgrid=False,
        showticklabels=False
    ),
    xaxis=dict(title=""),
    barmode='group',
    legend=dict(
        x=1, y=1,
        xanchor='right', yanchor='top',
        bgcolor='rgba(255,255,255,0)',
        bordercolor='rgba(255,255,255,0)',
        borderwidth=0
    )
)

fig.show()

Усиление успешных кампаний:
*Кампании performance.max_digitalmarket_ru_DE (112 сделок), youtube_shorts (53 сделки) и 02.07.23wide_DE (52 сделки) показывают наилучшие результаты по количеству сделок. Увеличьте бюджет на эти кампании и протестируйте расширение их аудитории для дальнейшего роста объема сделок.*
Оптимизация кампаний с высоким средним чеком:
Кампании blog2_DE, ASA_de_DE имеют высокий средний чек, но низкий объем сделок. *Курсив* *Проведите анализ их таргетинга и попробуйте увеличить количество сделок через снижение барьеров входа (например, пробные уроки) или дополнительные стимулы (скидки), сохраняя высокий средний чек.*
Пересмотр неэффективных кампаний:
*Кампании с минимальным количеством сделок и низкой средней суммой следует либо приостановить, либо переработать. Проведите A/B-тестирование новых форматов рекламы или таргетинга, чтобы понять, можно ли повысить их эффективность, либо перераспределите бюджет на более успешные кампании.*
Балансировка количества и суммы сделок:
*Для кампаний с высоким количеством сделок протестируйте повышение среднего чека через кросс-продажи или премиум-услуги. Для кампаний с высоким чеком увеличьте объем сделок через более широкий охват аудитории.*

**Проанализируем расходы и доходы по кампаниям и оценим их эффективность через показатель ROI.**

In [170]:
spend_aggregated = spend_df.groupby('Campaign')['Spend'].sum().reset_index()

deals_aggregated = (
    deals_df[deals_df["Stage"] == "Payment Done"]
    .groupby("Campaign")["Offer Total Amount (€)"]
    .sum()
    .reset_index()
)

merged_finance = pd.merge(
    spend_aggregated,
    deals_aggregated,
    on="Campaign",
    how="outer"
)

finance_analysis = pd.DataFrame({
    "Campaign": merged_finance["Campaign"],
    "Total Spend (€)": merged_finance["Spend"].round(2),
    "Total Revenue (€)": merged_finance["Offer Total Amount (€)"].round(1)
})

finance_analysis = finance_analysis[finance_analysis["Total Spend (€)"] > 50]

finance_analysis["ROI (%)"] = (
    (finance_analysis["Total Revenue (€)"] - finance_analysis["Total Spend (€)"])
    / finance_analysis["Total Spend (€)"]
    * 100
).round(1)

finance_analysis = finance_analysis.sort_values("ROI (%)", ascending=False)

print(finance_analysis.to_string())

                       Campaign  Total Spend (€)  Total Revenue (€)  ROI (%)
86                   web2408_DE           189.00            33000.0  17360.3
36              20.05.24wide_DE            76.72            11000.0  14237.9
28          18.10.23wide_gos_DE           325.10            22000.0   6667.1
19    12.09.23interests_Uxui_DE          3751.47           229000.0   6004.3
5                 03.07.23women          4218.89           256000.0   5967.9
9                07.07.23LAL_DE          4198.83           243500.0   5699.2
3               02.07.23wide_DE          6911.52           396900.0   5642.6
57                     blog2_DE           773.00            44000.0   5592.1
40       24.09.23retargeting_DE          2816.64           146000.0   5083.5
8             05.09.2023wide_DE           244.51            11000.0   4398.8
6      04.07.23recentlymoved_DE          4520.41           199900.0   4322.2
18            12.07.2023wide_DE          9471.52           336900.0   3457.0

*Кампании показывают, что небольшой рекламный бюджет (например, 20.05.24wide_DE с €76.72) может давать значительный возврат.
performancemax_eng_DE потратила €34,183.45, но заработала всего €11000, что дало ROI -67%.*

*Возможно, неправильно учитываются доходы или расходы (ошибки в данных).*

# **Анализ платежей и продуктов:**


## 1. Изучите распределение типов оплаты и их влияние на успешность сделок.

In [ ]:
payment_summary = deals_df[deals_df['Payment Type'].notna()].groupby(
    ['Payment Type', 'Stage']
).size().unstack(fill_value=0).reset_index()

payment_summary.columns.name = None
payment_summary = payment_summary.rename(
    columns={payment_summary.columns[0]: 'Payment Type'}
)

payment_summary['Successful Deals'] = payment_summary['Payment Done']
payment_summary['Unsuccessful Deals'] = payment_summary.drop(
    ['Payment Type', 'Payment Done'], axis=1
).sum(axis=1)

payment_summary['Conversion Rate (%)'] = (
    payment_summary['Successful Deals'] /
    (payment_summary['Successful Deals'] + payment_summary['Unsuccessful Deals']) * 100
).round(1)

display(payment_summary)

,Payment Type,Call Delayed,Lost,Payment Done,Qualificated,Registered on Webinar,Waiting For Payment,Successful Deals,Unsuccessful Deals,Conversion Rate (%)
0,One Payment,2,15,112,3,0,2,112,134,45.5
1,Recurring Payments,9,79,248,2,1,1,248,340,42.2
2,Reservation,0,4,1,0,0,0,1,5,16.7


*Единоразовые платежи One Payment и подписки Recurring Payments – лучшие варианты для бизнеса. Reservation работает плохо и требует пересмотра.Возможно, стоит изменить условия или убрать этот способ оплаты. Высокий процент неуспешных сделок по Recurring Payments 340 случаев, возможно, клиенты отказываются из-за долгосрочных обязательств – стоит уточнить причину.*
*Оптимизация процесса оплаты может увеличить общую конверсию. Например, добавить более понятные условия или бонусы за быструю оплату*.

**Анализ успешности сделок и конверсии по типам оплаты**

In [ ]:
fig = px.bar(
    payment_summary,
    x='Payment Type',
    y=['Successful Deals', 'Unsuccessful Deals'],
    title="Количество успешных и неуспешных сделок по типам оплаты",
    labels={'value': 'Количество сделок', 'variable': 'Тип сделки'},
    barmode='stack',
    color_discrete_sequence=['#a8d0f5', '#dfd1ba']
)

fig.add_trace(go.Scatter(
    x=payment_summary['Payment Type'],
    y=payment_summary['Conversion Rate (%)'],
    name='Конверсия (%)',
    line=dict(color='#08306b', width=3),
    yaxis='y2'
))

for i, row in payment_summary.iterrows():
    fig.add_annotation(
        x=row['Payment Type'],
        y=row['Conversion Rate (%)'] + 3,
        text=f"{row['Conversion Rate (%)']}%",
        showarrow=False,
        font=dict(size=12, color="black"),
        align="center",
        yref="y2"
    )

fig.update_layout(
    legend=dict(
        x=1,
        y=1.1,
        traceorder='normal',
        title='Тип сделки',
        font=dict(size=12),
        bgcolor='white'

    ),
    yaxis2=dict(
        title='',
        overlaying='y',
        side='right',
        range=[0, 100],
        showticklabels=False
    ),
    xaxis_title='',
    yaxis_title='Количество сделок',
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()

O*ne Payment: Конверсия составляет 45.5%, что значительно выше, чем у других типов оплаты.Это может указывать на то, что клиенты предпочитают единоразовые платежи, так как они проще и понятнее.*

*Recurring Payments (Регулярные платежи):Конверсия составляет 42,2%, что ниже, чем у единоразовых платежей.Это может быть связано с тем, что регулярные платежи требуют большего доверия со стороны клиентов и более сложной настройки.*

*Reservation (Бронирование):Наименьшее количество успешных сделок.Возможно, клиенты неохотно используют бронирование из-за необходимости внесения предоплаты или других ограничений.*

**Распределение суммы полученных средств по типам оплаты и стадиям сделки**

In [ ]:
payment_revenue_by_stage = deals_df.groupby(['Payment Type', 'Stage'])[
    'Offer Total Amount (€)'].sum().reset_index()

fig = px.bar(
    payment_revenue_by_stage,
    x='Payment Type',
    y='Offer Total Amount (€)',
    color='Stage',
    title="Сумма полученных средств по типам оплаты и стадиям",
    labels={'Offer Total Amount (€)': 'Сумма полученных средств (€)',
            'Payment Type': 'Тип оплаты', 'Stage': 'Стадия'},
    color_discrete_sequence=['#08306b', '#ffb347', '#85a8df', '#cfb09b',
                             '#9c7648']
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Тип оплаты',
    yaxis_title='Сумма полученных средств (€)',
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis_tickangle=-45,
    barmode='group',
)

fig.show()

*Регулярные платежи (Recurring Payments) приносят наибольшую сумму выручки (1 млн €), что делает их ключевым источником дохода.*
*Значительная сумма потерь (0,5 млн €) у регулярных платежей требует анализа причин (например, отказы от подписок, задержки платежей) и оптимизации процесса.*
*Reservation показывает минимальную активность и практически не генерирует доход, что может указывать на необходимость пересмотра стратегии или улучшения процесса.*
 One Payment демонстрирует стабильный результат с минимальными потерями, что делает их надежным источником дохода.
*Нужно сосредоточиться на минимизации потерь у регулярных платежей и активизации процесса резервирования для повышения общей эффективности.*

## 2. Проанализируйте популярность и успешность различных продуктов и типов
обучения.


In [ ]:
sunburst_data = deals_df.dropna(subset=['Product', 'Education Type']).groupby(
    ['Product', 'Education Type']
)['Offer Total Amount (€)'].sum().reset_index()

payment_done_deals_df = deals_df[deals_df['Stage'] == 'Payment Done']

grouped_deals = payment_done_deals_df.groupby(['Product', 'Education Type']).\
                size().reset_index(name='Count')

colors = ['#a8d0f5', '#3e6aa4', '#dfd1ba']
colors_1 = ['#85a8df', '#08306b', '#9c7648' ]
fig1 = px.sunburst(
    sunburst_data,
    path=['Product', 'Education Type'],
    values='Offer Total Amount (€)',
    title='Распределение выручки по продуктам и типам обучения',
    color='Product',
    color_discrete_sequence=colors,
    maxdepth=2,
    branchvalues='total'
)

fig1.update_traces(
    texttemplate='%{label}<br>%{percentParent:.1%}<br>%{value}',
    insidetextorientation='radial'
)

fig1.update_layout(
    title_x=0.5,
    margin=dict(t=50, b=0, l=0, r=0),
    height=800,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig2 = px.sunburst(
    grouped_deals,
    path=['Product', 'Education Type'],
    values='Count',
    title="Количество успешных сделок по продуктам и типам обучения",
    color='Product',
    color_discrete_sequence=colors_1,
    hover_data={'Product': True, 'Education Type': True, 'Count': True}
)

fig2.update_traces(
    texttemplate='%{label}<br>%{percentParent:.1%}<br>%{value}',
    insidetextorientation='radial'
)

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Распределение выручки', 'Количество успешных сделок'),
    specs=[[{'type': 'sunburst'}, {'type': 'sunburst'}]]
)

for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

for trace in fig2.data:
    fig.add_trace(trace, row=1, col=2)

fig.update_layout(
    title="Сравнение распределения выручки и количества сделок по продуктам и типам обучения",
    title_x=0.5,
    height=800,
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=False
)

fig.show()

*В Digital Marketing и UX/UI Design утренние курсы не только имеют больше успешных сделок, но и значительно большие суммы, что может говорить о более высоком спросе и ценности этих курсов среди клиентов.*
*В Web Developer вечерние курсы имеют минимальное количество успешных сделок (1 сделка) и очень низкую сумму (2,000 €), что может свидетельствовать о слабой востребованности вечерних курсов по этому продукту*.

*Digital Marketing и UX/UI Design показывают высокую активность в утреннее время с большими суммами и количеством сделок, что свидетельствует о востребованности этих курсов в утренние часы.*
*Web Developer имеет крайне низкие результаты по вечерним курсам, что требует внимания, возможно, для улучшения маркетинга и предложений по вечерним курсам или изменения времени их проведения для повышения интереса*.

**РЕКОМЕНДАЦИИ**

***1. Увеличение количества звонков в периоды спада***

Проблема: Снижение количества звонков и сделок в определенные периоды

Увеличить количество звонков в периоды спада, чтобы поддерживать количество сделок на более высоком уровне. Например, внедрить дополнительные мотивационные программы для менеджеров.
Провести дополнительные маркетинговые кампании в периоды спада (например, увеличить бюджет на рекламу в июле), чтобы поддерживать приток лидов.

***2. Улучшение квалификации лидов на ранних стадиях ***

Проблема: Высокий уровень потерь на ранних стадиях (LOST).

Внедрить более строгий процесс квалификации лидов на стадии NEW LEAD, чтобы отсеивать нецелевых клиентов до начала активной работы с ними.
Использовать автоматизированные инструменты (например, CRM-системы с фильтрами) для оценки лидов по ключевым критериям (интерес, бюджет, готовность к покупке).
Провести обучение менеджеров, у которых высокий уровень LOST, чтобы улучшить их навыки квалификации и первичного контакта с клиентами.

***3. Увеличение конверсии на поздних стадиях воронки продаж***

Проблема: Низкая конверсия на поздних стадиях, таких как PAYMENT DONE

Предложить гибкие способы оплаты (рассрочка, скидки за быструю оплату), чтобы уменьшить нерешительность клиентов.
Усилить работу менеджеров на стадии NEED TO CALL - SALES, чтобы быстрее переводить лидов на стадию оплаты. Например, использовать скрипты продаж, ориентированные на закрытие сделки.

***4. Оптимизация кампаний для повышения эффективности***

Проблема: Низкая конверсия и низкая средняя сумма сделки в кампаниях с большим количеством лидов

Сфокусироваться на кампаниях с высоким потенциалом, которые показывают умеренное количество сделок, высокую среднюю сумму сделки и больше сделок на поздних стадиях (PAYMENT DONE). Увеличить бюджет на вебинары, так как они привлекают более качественных лидов.
Для кампаний с большим количеством лидов, но низкой конверсией, пересмотреть таргетинг и формат рекламы, чтобы привлекать более целевую аудиторию.
Изучить успешные кампании с высокой конверсией и применить их подходы (например, таргетинг, формат рекламы) к менее эффективным кампаниям.

***5. Перераспределение нагрузки между менеджерами***

Проблема: Неравномерное распределение нагрузки и низкая конверсия у топ-менеджеров

Перераспределить нагрузку между менеджерами, чтобы топ-менеджеры могли сосредоточиться на меньшем количестве лидов и повысить конверсию.
Передать часть лидов менеджерам с более высокой конверсией, особенно для кампаний с высоким потенциалом, таких как webinar1906_DE.
Внедрить систему ротации, чтобы менеджеры с меньшей нагрузкой могли обрабатывать больше лидов.

***6. Оптимизация типов оплаты:***

Проблема: Низкая конверсия и значительные потери, особенно у регулярных платежей Recurring Payments и резервирования Reservation, что приводит к снижению общей эффективности системы.

Сфокусироваться на однократных платежах One Payment.
Улучшить регулярных платежей Recurring Payments. Внедрить улучшения: автоматизированные напоминания о платежах, гибкие планы подписок и персонализированную поддержку клиентов.
Пересмотр процесса резервирования Reservation. Провести аудит процесса: упростить шаги, добавить стимулы (например, бонусы за завершение оплаты) и протестируйте новые подходы на продуктах с высоким потенциалом.

***7. Увеличение спроса и ценности курсов***

Проблема: Низкая востребованность вечерних курсов

Оптимизация времени проведения вечерних курсов:Рассмотреть возможность переноса некоторых курсов на более позднее время или в выходные, чтобы привлечь студентов, которые не могут заниматься в вечерние часы буднего дня.
Разработать гибкий график, например, вечерние курсы дважды в неделю или по требованию, чтобы они стали более удобными для студентов.
Промо-акции для вечерних курсов:
Запустить акции и скидки для вечерних курсов с целью привлечь студентов. Например, скидки на первый курс или пакетные предложения.
Предложить бонусы и стимулы для студентов, записавшихся на вечерние курсы, такие как дополнительные консультации, доступ к эксклюзивным материалам или тренингам.

***8. Усовершенствование системы сбора данных CRM***

Проблема: Данные в CRM-системе содержат много пропусков , что затрудняет анализ и принятие решений.

Внедрить автоматизированные проверки и валидацию данных на этапе их ввода (например, обязательные поля для ключевых параметров, таких как источник лида или стадия сделки), чтобы минимизировать пропуски. Провести оптимизацию существующих данных, заполнив пропуски с помощью анализа исторических данных или запросов к клиентам. Обучить сотрудников правильному заполнению CRM, включая регулярные тренинги и контроль качества.